In [ ]:
# TODO documentation
# Based on https://github.com/seanrjohnson/protein_scoring/blob/main/colab_notebooks/Protein_Metrics.ipynb

In [1]:
# Configuration

# dataset to be filtered for activation cadidates
input_file = '../generated_sequences/basemodel-RITA_l-generated.fasta'

# the test dataset to compare ESM-1v scores against
testset_file = '../datasets/InterProUniprotPF03272prepared_test.fasta'

In [ ]:
# Install dependencies
import os

!pip -q install  git+https://github.com/facebookresearch/esm.git@e5e7b06b9a093706607c229ab1c5c9821806814d

if (not os.path.isdir('protein_gibbs_sampler')):
    !git clone https://github.com/seanrjohnson/protein_gibbs_sampler > /dev/null
    !cd protein_gibbs_sampler ; git checkout 7de11ff63f7dca3cb52ef84109152ac8f988b7a0 2>&1 > /dev/null ; yes | pip install -q --no-dependencies . > /dev/null

!yes | pip -q install  biopython~=1.79 pytest-datadir~=1.4.1 > /dev/null

In [ ]:
# Read the input sequences to be evaluated
from Bio import SeqIO

dataset = list(SeqIO.parse(input_file, 'fasta'))

In [ ]:
# Perform the 'Quality Checks'
from quality_checks import main as quality_checks

dataset = quality_checks(dataset)

In [ ]:
from esm_likelihood import get_esm_likelihood_scores
from Bio import SeqIO

testset = list(SeqIO.parse(testset_file, 'fasta'))

# filter sequences with more than 1024 amino acids because ESM can't handle longer sequences
testset = [seq for seq in testset if len(seq) <= 1024]

# Calculate cutoff (10th percentile of ESM likelihood scores for the test set)
testset_df = get_esm_likelihood_scores(testset)
cutoff = testset_df['score'].quantile(q=0.1)
cutoff

In [ ]:
# Filter sequences with scores below cutoff
df = get_esm_likelihood_scores(dataset)

filtered = df[df['score'] > cutoff]
ids = list(filtered['id'])